In [ ]:
def read_data(path):
    S = []
    costs = [0]
    first = True
    with open(path, "r") as f:
        csv.reader(f, delimiter=" ")
        
        for line in f:
            info = line.split(" ")
            if first:
                first = False
                n_sets = info[0]
                n_elems = info[1]
            else:
                s = set()
                costs.append(int(info[0]))
                info.pop(0)
                for e in info:
                    e.replace("\n", "")
                    s.add(int(e))
                S.append(s)
    return int(n_elems), int(n_sets), S, costs


In [ ]:
def init():
    line.set_data([], [])
    return line,

def animate(i, every_sol,ax,tours,m):
    
    t = (i*tours)//(tours*m-1) + 1
    
    ant = i%m + 1
    
    title = "Tour: " + str(t) + "     Ant: " + str(ant) 
    ax.set_title(title, va='top')
    sol = every_sol[i]
    
    x = [i+1 for i in range(n_sets)]
    y = []
    
    for i in range(1, n_sets + 1):
        if i in sol:
            y.append(1)
        else:
            y.append(0)
    
    line.set_data(x, y)
    return line,

In [ ]:
def save_animation(tours, m, every_sol, maxx):
    Writer = writers['ffmpeg']
    writer = Writer(fps=20, metadata=dict(artist='Me'), bitrate=1000)

    global line

    fig = plt.figure(figsize=(25,10))
    ax = plt.axes(xlim=(0,maxx+2), ylim=(-1, 2))
    
    
    line, = ax.plot([],[], color='white', linewidth = 3, 
         marker='o', markerfacecolor='green', markersize=12)


    anim = FuncAnimation(fig, func=animate, blit=True, init_func=init, frames=len(every_sol), repeat=False, fargs=(every_sol,ax,tours,m),interval=100)
    
    name = "SetCover-"+str(tours) + "tours-" + str(m) +"ants"
    anim.save(name + '.mp4', writer=writer, dpi=100)

In [ ]:
def s_to_matrix(n_elems, n_sets, S):
    
    # Inicializar matriz todo ceros
    matrix = np.zeros((n_sets+1, n_elems+1))
    
    # Los elementos van de 1 a n_elems
    
    col = 1
    for s in S:
        for e in s:
            matrix[e][col] = 1
        col += 1
        
    return matrix

# <center> ACO para Set Cover </center>

In [ ]:
import os
import csv
import numpy as np
import math
import matplotlib.pyplot as plt 
import matplotlib
from matplotlib.animation import FuncAnimation
from matplotlib.animation import writers

Coger un conjunto *j* es factible si cubre el elemento *i* e *i* no está ya cubierto

In [ ]:
def isFeasible(i, j, matrix, currently_uncovered):
    return (matrix[i][j] == 1) and (i in currently_uncovered)

Calcula cuántos elementos nuevos cubre ej conjunto j

In [ ]:
def calculate_n_new_covered(n_elems, matrix, j, currently_uncovered):
    n_new_covered = 0
    for i in range(1, n_elems+1):
        # Si el conjunto j cubre el elemento i, y el elemento i no está ya cubierto, se suma uno
        if (matrix[i][j] == 1) and (i in currently_uncovered):
            n_new_covered += 1
    return n_new_covered

Calcula la probabilidad de que el conjunto *j* sea el elegido para cubrir el elemento *i*. Para ello, utiliza el sendero de feromonas hasta el momento y la heurística eta (número de nuevos elementos que cubre *j* / coste de *j*), la fórmula es la siguiente:
<img src="img/p_scp.png" alt="Drawing" style="width: 500px;"/>

In [ ]:
def calculate_probability(i, j, costs, pheromone_trail, matrix, n_elems, n_sets, currently_uncovered):
    # psi_j = número de nuevos elementos que puede cubrir el conjunto j
    psi_j = calculate_n_new_covered(n_elems, matrix, j, currently_uncovered)
    
    # eta = heurística
    eta = psi_j / costs[j]
    
    # Importancia relativa de la heurística con respecto a la feromona
    beta = 1 # >= 0
    
    numerator = pheromone_trail[j] * pow(eta, beta)
    
    denominator = 0
    
    # Para cada conjunto q que cubre el elemento i
    for q in range(1, n_sets+1):
        if isFeasible(i, q, matrix, currently_uncovered):
            psi_q = calculate_n_new_covered(n_elems, matrix, q, currently_uncovered)
            denominator += pheromone_trail[j] * pow((psi_q / costs[q]),beta)
    
    return numerator / denominator

Se actualiza la mejor solución si es que se ha obtenido una de menor coste que la mejor hasta el momento. Además, se actualizan los límites de valor de feromona tmin tmax según las siguientes fórmulas:
<img src="img/tmax.png" alt="Drawing" style="width: 250px;"/>
<img src="img/tmin.png" alt="Drawing" style="width: 130px;"/>

In [ ]:
def updateBestSolution(sol, best_cost, best_solution, evaporation_rate, costs, epsilon, phmax, phmin):
    # Actualizar nueva mejor solución
    total_cost = 0
    for j in sol:
        total_cost += costs[j]
    
    if total_cost < best_cost:
        best_cost = total_cost
        best_solution = sol
    
        # Actualizar phmin y phmax
        phmax = 1 / ((1 - evaporation_rate) * best_cost)     # tmax
        phmin = epsilon*phmax                                # tmmi
    
    return best_solution, best_cost, phmax, phmin

Devuelve los elementos que cubre el conjunto s

In [ ]:
def covered_by(s, n_elems, matrix):
    elems = []
    for elem in range(1, n_elems+1):
        if matrix[elem][s] == 1:
            elems.append(elem)
    return elems

Una hormiga construye una solución, empezando desde una solución vacía *solution_construction*.


Se añaden conjuntos *chosen_set* iterativamente a *solution_construction* hasta que todos los elementos queden cubiertos: 

1. Se elige un elemento que debe ser cubierto (orden: aleatorio, natural o ascendiente por coste), en esta implementación se ha elegido el orden natural

2. Se calcula la probabilidad de cada conjunto j de cubirir el elemento i

3. Se elige el conjunto *chosen_set* aleatoriamente (probabilidad)

4. Se añade *chosen_set*  a *solution_construction*

In [ ]:
def constructAntSolution(matrix, n_sets, n_elems, pheromone_trail):
    solution_construction = []
    currently_covered = []
    currently_uncovered = [x for x in range(1, n_elems+1)]
    
    for i in range(1, n_elems+1): # 1.  Elegir un elemento a cubrir
        p = []
        a = []
        if i in currently_uncovered:
            for j in range(1, n_sets+1): 
                if isFeasible(i, j, matrix, currently_uncovered): # Para cada conjunto factible
                    # 2.   Calcular la probabilidad de elegir el conjunto j
                    p.append(calculate_probability(i, j, costs, pheromone_trail, matrix, n_elems, n_sets, currently_uncovered))
                    a.append(j)
        
            #  3.   Elegir aleatoriamente el conjunto
            chosen_set = np.random.choice(a, 1, p=p)[0]
            
            # Actualizar elementos cubiertos y no cubiertos
            for elem in covered_by(chosen_set, n_elems, matrix): 
                if elem not in currently_covered:
                    currently_covered.append(elem)
                    currently_uncovered.remove(elem)
        
            # 4.   Añadir el conjunto seleccionado a la solución
            solution_construction.append(chosen_set)

    return solution_construction

Se actualiza el sendero de feromonas basándose en la fórmula:
<img src="img/ph_scp.png" alt="Drawing" style="width: 250px;"/>
donde:

ⲣ = evaporación de la feromona

Δt = cantidad de feromona depositada:

[tmin, tmax] = intervalo límite de valor de feromona

cq = coste del conjunto q

Sgb = mejor solución global hasta el momento (conjunto de conjuntos)

In [ ]:
def updatePheromone(pheromone_trail, n_sets, best_solution, costs, phmax, phmin, evaporation_rate):
    
    # Evaporation
    for j in range(1, n_sets + 1):
        pheromone_trail[j] *= (1 - evaporation_rate)
    
    # Para cada conjunto de la mejor solución, se le suma el valor de la feromona (limitado por phmax y phmin)
    for q in best_solution:
        phq = pheromone_trail[q] + (1 / costs[q])
        if phq > phmax:
            pheromone_trail[q] = phmax
        elif phq < phmin:
            pheromone_trail[q] = phmin
        else:
            pheromone_trail[q] = phq

Algoritmo general del ACO:
<img src="img/aco.png" alt="Drawing" style="width: 300px;"/>

In [ ]:
def aco(matrix, m, n_elems, n_sets, tours, costs):
    
     # 1.   Ajustado en el método que llama a este método (m, n, tours)
    
    pheromone_trail = []
    
    best_solution = []
    best_cost = math.inf

    epsilon = 0.2 # 0 < epsilon < 1 coeficiente de relación
    phmax = math.inf
    phmin = math.inf
    
    '''every_sol = [] # Para animación'''
    evaporation_rate = 0.3
    
    # 2.   Inicializar sendero de feromonas, todo a 0
    pheromone_trail = [0.000000001]*(n_sets+1)

    for t in range(tours):
        solutions = []
       
        # Cada hormiga construye una solución, que se guarda en solutions
        for ant in range(m):
            # 3.   Construcción de la solución por la hormiga ant
            sol = constructAntSolution(matrix, n_sets, n_elems, pheromone_trail)
            '''every_sol.append(sol)'''
            
            solutions.append(sol)
            
            # Actualizar mejor solución
            best_solution, best_cost, phmax, phmin = updateBestSolution(sol, best_cost, best_solution, evaporation_rate, costs, epsilon, phmax, phmin)
        

        # 4.   Actualización del sendero de feromonas utilizando las soluciones que han generado las m hormigas
        updatePheromone(pheromone_trail, n_sets, best_solution, costs, phmax, phmin, evaporation_rate)
        
       
    print(best_solution)
    print(best_cost)
    '''save_animation(tours, m, every_sol, n_sets)'''

In [ ]:
path = os.path.join(os.getcwd(), 'dataset', 'scp', "set_cover_100.txt")
n_elems, n_sets, S, costs = read_data(path)
matrix = s_to_matrix(n_sets, n_elems, S)

m = 15 # number of ants

tours = 50 # number of tours
iterations = 1


for it in range(iterations):
    
    #start=datetime.now()
    
    aco(matrix, m, n_elems, n_sets, tours, costs)
    
    #time = datetime.now()-start

    m+=5